In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import datetime

/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


<img src="optimizer_graph.png" width="200px">

## Reading Data

Data from https://archive.ics.uci.edu/ml/datasets/auto+mpg

In [2]:
autompg = pd.read_csv('auto_mpg.csv')
# Extract only the displacement and mpg column
autompg_disp = autompg['displacement'].astype(float)
autompg_mpg = autompg['mpg'].astype(float)

# Scale data
autompg_disp = autompg_disp.apply(lambda x:x/100)
print(autompg_disp.describe())
autompg_mpg = autompg_mpg.apply(lambda x: x/100)
print(autompg_mpg.describe())

count    398.000000
mean       1.934259
std        1.042698
min        0.680000
25%        1.042500
50%        1.485000
75%        2.620000
max        4.550000
Name: displacement, dtype: float64
count    398.000000
mean       0.235146
std        0.078160
min        0.090000
25%        0.175000
50%        0.230000
75%        0.290000
max        0.466000
Name: mpg, dtype: float64


In [5]:
# Convert numpy data series to numpy array
train_X = np.asarray(autompg_disp)
train_Y = np.asarray(autompg_mpg)
n_samples = train_X.shape[0]

In [6]:
# get batch of data
def getbatch(xval, yval, arraylength, batchsize=30):
    count = 0 
    while count < arraylength/batchsize:
        randstart = random.randint(0, arraylength-batchsize-1)
        count += 1
        yield (xval[randstart:randstart+batchsize], yval[randstart:randstart+batchsize])

# Test
for i in getbatch(train_X, train_Y, n_samples):
    print(i)

(array([ 1.13 ,  0.98 ,  2.32 ,  2.25 ,  2.5  ,  2.5  ,  2.32 ,  3.5  ,
        4.   ,  3.51 ,  3.18 ,  3.83 ,  4.   ,  4.   ,  2.58 ,  1.4  ,
        2.5  ,  2.5  ,  1.22 ,  1.16 ,  0.79 ,  0.88 ,  0.71 ,  0.72 ,
        0.97 ,  0.91 ,  1.13 ,  0.975,  0.97 ,  1.4  ]), array([ 0.25,  0.25,  0.19,  0.16,  0.17,  0.19,  0.18,  0.14,  0.14,
        0.14,  0.14,  0.12,  0.13,  0.13,  0.18,  0.22,  0.19,  0.18,
        0.23,  0.28,  0.3 ,  0.3 ,  0.31,  0.35,  0.27,  0.26,  0.24,
        0.25,  0.23,  0.2 ]))
(array([ 0.91,  1.51,  1.73,  1.73,  1.51,  0.98,  0.89,  0.98,  0.86,
        1.51,  1.4 ,  1.51,  2.25,  0.97,  1.34,  1.2 ,  1.19,  1.08,
        0.86,  1.56,  0.85,  0.9 ,  0.9 ,  1.21,  1.46,  0.91,  0.85,
        0.97,  0.89,  1.68]), array([ 0.373,  0.284,  0.288,  0.268,  0.335,  0.415,  0.381,  0.321,
        0.372,  0.28 ,  0.264,  0.243,  0.191,  0.343,  0.298,  0.313,
        0.37 ,  0.322,  0.466,  0.279,  0.408,  0.443,  0.434,  0.364,
        0.3  ,  0.446,  0.409,  0.3

## Setting hyper-parameters

In [7]:
learning_rate = 0.003
n_epochs = 3000
display_step = 100

## Defining X and Y as placeholders

In [8]:
# Define placeholders and variables
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

## Initializing Weights and Biases

In [9]:
W = tf.Variable(np.random.randn())
b = tf.Variable(np.random.randn())

## Defining model

predicted is X*W+b

In [10]:
pred = tf.add(tf.multiply(X, W), b)

## Define cost and optimizer

In [11]:
# The cost function is ((predicted-actual)^2)/2*n_samples. 
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Add all ops that need to be initialized
# The initilization needs to be run only after session is created
# as in session.run(init) below.
init = tf.global_variables_initializer()

In [12]:
# This cell is new compared to the previous linear regression code
# Summary writer and its configuration
writer = tf.summary.FileWriter("./logs", graph=tf.get_default_graph())
tf.summary.scalar("Cost", cost)
merged_summary_op = tf.summary.merge_all()

## Running the session 
This will run the graph and use all the tensors that were previously defined

In [13]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for (x, y) in getbatch(train_X, train_Y, n_samples):
            # In this line, we added write merged_summary that will write the cost to tensorboard
            _, costval, merged_summary = sess.run([optimizer, cost, merged_summary_op], 
                                                  feed_dict={X: x, Y: y})
        writer.add_summary(merged_summary, epoch)
        # Every few display step, calculate and print cost
        if (epoch+1) % display_step == 0:
            c = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
            print("Epoch = {:04d} and cost = {:.9f}".format(epoch+1, c))

    # At the end of the training, print the cost and W and b
    training_cost = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
    finalW = sess.run(W)
    finalb = sess.run(b)
    print("The final W = %0.4f and b = %0.4f" %(finalW, finalb))
    
writer.close()

Epoch = 0100 and cost = 0.031425934
Epoch = 0200 and cost = 0.027423650
Epoch = 0300 and cost = 0.024433196
Epoch = 0400 and cost = 0.021817476
Epoch = 0500 and cost = 0.019474516
Epoch = 0600 and cost = 0.017382778
Epoch = 0700 and cost = 0.015555291
Epoch = 0800 and cost = 0.013924750
Epoch = 0900 and cost = 0.012488062
Epoch = 1000 and cost = 0.011209786
Epoch = 1100 and cost = 0.010059898
Epoch = 1200 and cost = 0.009045811
Epoch = 1300 and cost = 0.008146546
Epoch = 1400 and cost = 0.007367916
Epoch = 1500 and cost = 0.006653674
Epoch = 1600 and cost = 0.006028199
Epoch = 1700 and cost = 0.005462651
Epoch = 1800 and cost = 0.004976470
Epoch = 1900 and cost = 0.004542189
Epoch = 2000 and cost = 0.004143090
Epoch = 2100 and cost = 0.003796492
Epoch = 2200 and cost = 0.003487252
Epoch = 2300 and cost = 0.003213042
Epoch = 2400 and cost = 0.002974840
Epoch = 2500 and cost = 0.002761696
Epoch = 2600 and cost = 0.002576479
Epoch = 2700 and cost = 0.002412339
Epoch = 2800 and cost = 0.00

## Opening tensorboard

In the command line, type "tensorboard --logdir=./logs"
This will launch a local webserver at port 6006. Visit http://localhost:6006 in a browser. Then click on the label 'Cost' to see the graph.  You can make the graph larger by clicking on the rectangular button at the bottom left corner of the graph. You can zoom in to the graph by drawing a box around the region to zoom in. You can zoom out by double clicking the graph.

NOTE 1: Please make sure that you ran this code to completion and that the log folder exist. 

NOTE 2: Make sure that you are in the directory that has the logs folder when you execute the tensorboard command. 

On Windows, visiting http://localhost:6006 returns an error. Instead use the link provided by Tensorboard in the terminal. On the other hand, some people using Mac have experienced problem using the link provided by Tensorboard.  In such cases, use localhost:6006 instead. 